In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize a BigQuery client
# In Colab Enterprise, this will automatically authenticate using your project's credentials.
client = bigquery.Client()

# Define your BigQuery table reference
project_id = 'youtube-content-optimize'
dataset_id = 'streaming_data'
table_id = 'youtube_video_stats'

# Construct your SQL query
# You can select all columns, or specify only the ones you need.
# Add a WHERE clause, ORDER BY, or LIMIT as necessary for your analysis.
query = f"""
    SELECT *
    FROM `{project_id}.{dataset_id}.{table_id}`
    -- Optionally, add a LIMIT for testing or if you only need a subset of data
    -- LIMIT 1000
"""

# Run the query and convert the results to a Pandas DataFrame
# .to_dataframe() is a convenient method for direct conversion.
try:
    df = client.query(query).to_dataframe()
    print(f"Successfully loaded {len(df)} rows into a Pandas DataFrame.")
    print(df.head()) # Display the first few rows of the DataFrame
except Exception as e:
    print(f"An error occurred: {e}")



Successfully loaded 200 rows into a Pandas DataFrame.
      video_id                                              title  \
0  _n_XmyEhSNY                                   PRESIDENT - RAGE   
1  zHyt69mJnj8        Friday Night Funkin - Mobile Launch Trailer   
2  qebbMeyU_C4  Mellow Rackz & Youngboy Never Broke Again - Gu...   
3  DVuiem6wG1s  $0 to $1 Trillion Using ONLY PRISMATIC Seeds i...   
4  xxNFkHzwAA0  College Graduates VS Dropouts Debate: Is Colle...   

           published_at      channel_title                channel_id  \
0  2025-07-17T23:00:06Z          PRESIDENT  UCCAxEuKZzOWyUZHFzuoTcjA   
1  2025-07-21T15:09:32Z  FridayNightFunkin  UC9RfC_6a8t8MaN9GwFSjG1A   
2  2025-07-19T13:04:04Z       Mellow Rackz  UC8adG-r_HDAI9edf6WR66fA   
3  2025-07-19T03:10:54Z        peach plays  UCubCKTdlEsDrWGIlMEgX_Sg   
4  2025-07-18T20:30:11Z               FaZe  UCTevXUV9L6_hgvFlLDMmf1w   

          channel_published_at       channel_name  subscriber_count  \
0   2025-01-01T14:14:54.150

In [ ]:
# Ensure both are timezone-naive
df['publishedAt'] = pd.to_datetime(df['published_at'], errors='coerce').dt.tz_localize(None)

today = pd.to_datetime('today').tz_localize(None)

df['days_since_published'] = (today - df['publishedAt']).dt.days

In [ ]:
# Avoid division by zero
import numpy as np
df['channel_published_at'] = pd.to_datetime(df['channel_published_at'], errors='coerce').dt.tz_localize(None)
today = pd.to_datetime('today').tz_localize(None)
df['channel_published_at'] = (today - df['channel_published_at']).dt.days
df['channel_age_days'] = df['channel_published_at'].replace(0, np.nan)

# subscriber_growth_rate = subscriberCount / channel_age_days
df['subscriber_growth_rate'] = df['subscriber_count'] / df['channel_age_days']

# upload_frequency = videoCount / channel_age_days
df['upload_frequency'] = df['video_count'] / df['channel_age_days']

# views_per_upload = viewCount / videoCount
df['views_per_upload'] = df['view_count'] / df['video_count']

# engagement_per_upload = (likeCount + commentCount) / videoCount
df['engagement_per_upload'] = (df['like_count'] + df['comment_count']) / df['video_count']

# subscriber_per_upload = subscriberCount / videoCount
df['subscriber_per_upload'] = df['subscriber_count'] / df['video_count']

# Replace infinite and NaN values with 0 for downstream clustering/regression
df = df.replace([np.inf, -np.inf], np.nan)
df.fillna(0, inplace=True)

In [ ]:
df['log_subscriber_growth_rate'] = np.log1p(df['subscriber_growth_rate'])
df['log_upload_frequency'] = np.log1p(df['upload_frequency'])
df['log_views_per_upload'] = np.log1p(df['views_per_upload'])
df['log_engagement_per_upload'] = np.log1p(df['engagement_per_upload'])
df['log_subscriber_per_upload'] = np.log1p(df['subscriber_per_upload'])

In [ ]:
import re

def safe_parse_tags(tag_str):
    if isinstance(tag_str, list):
        return tag_str
    if not isinstance(tag_str, str):
        return []
    # Match content inside brackets and split by comma
    if tag_str.startswith("[") and tag_str.endswith("]"):
        try:
            inner = tag_str[1:-1]
            tags = re.split(r',\s*', inner)
            tags = [t.strip(" '\"") for t in tags if t.strip()]
            return tags
        except Exception:
            return []
    return []

df['tags_list'] = df['tags'].apply(safe_parse_tags)
df['tags_joined'] = df['tags_list'].apply(lambda tags: ' '.join(tags))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=300)
tags_tfidf = vectorizer.fit_transform(df['tags_joined'])


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.sparse import hstack

# 1. Choose numeric features and scale them
numeric_features = ['view_count', 'duration_seconds', 'log_subscriber_growth_rate', 'log_upload_frequency', 'log_views_per_upload', 'log_engagement_per_upload', 'log_subscriber_per_upload', 'days_since_published']
scaled_numeric = StandardScaler().fit_transform(df[numeric_features])

# 2. Combine numeric and text features
X_combined = hstack([scaled_numeric, tags_tfidf])  # use sparse hstack for efficiency

# 3. Apply PCA for dimensionality reduction
pca = PCA(n_components=8) # Choose the number of components
X_reduced = pca.fit_transform(X_combined.toarray()) # Convert to dense array for PCA

# 4. Run KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(X_reduced)


In [ ]:
cluster_summary = df.groupby('cluster')[numeric_features].mean()
print(cluster_summary)

             view_count  duration_seconds  log_subscriber_growth_rate  \
cluster                                                                 
0        1006370.293333           1259.12                    0.158288   
1              355232.6            655.40                    3.467877   
2        4242935.083333           1110.25                    0.720471   

         log_upload_frequency  log_views_per_upload  \
cluster                                               
0                    0.003864              4.536429   
1                    0.222323              4.097647   
2                    0.002069              8.254914   

         log_engagement_per_upload  log_subscriber_per_upload  \
cluster                                                         
0                         1.263215                   5.053408   
1                         0.931712                   4.898361   
2                         5.143069                   8.802026   

         days_since_published  


In [ ]:
#number of rows in a cluster
df['cluster'].value_counts()

,count
cluster,
2,120
0,75
1,5


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

df['log_view_count'] = np.log1p(df['view_count'])
df['log_duration_seconds'] = np.log1p(df['duration_seconds'])

# === 1. Select log-transformed predictor variables ===
X = df[['log_duration_seconds', 'log_subscriber_growth_rate',
        'log_subscriber_per_upload', 'log_upload_frequency',
        'log_views_per_upload', 'log_engagement_per_upload']]

# === 2. Target variable: log_subscriber_growth_rate ===
y = df['log_view_count']

# === 3. Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === 4. Train linear regression model ===
model = LinearRegression()
model.fit(X_train, y_train)

# === 5. Predict and evaluate ===
y_pred = model.predict(X_test)

# Calculate MSE without squared=False, then take the square root for RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")

coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

print(coef_df)

df.to_csv("cleaned_youtube_data.csv", index=False)


RMSE: 1.119
R²: 0.319
                      Feature  Coefficient
3        log_upload_frequency     1.827799
4        log_views_per_upload     0.719673
5   log_engagement_per_upload    -0.533039
1  log_subscriber_growth_rate    -0.197604
0        log_duration_seconds    -0.146589
2   log_subscriber_per_upload     0.116599


In [ ]:
df.to_csv("cleaned_youtube_data.csv", index=False)
